In [1]:
from gensim.models.keyedvectors import KeyedVectors

word2VecModel = KeyedVectors.load_word2vec_format('D:/gn/GoogleNews-vectors-negative300.bin', binary=True)

angry = word2VecModel['angry']
disgust = word2VecModel['disgust']
fear = word2VecModel['fear']
happy = word2VecModel['happy']
neutral = word2VecModel['neutral']
sad = word2VecModel['sad']
surprise = word2VecModel['surprise']

weight = [angry, disgust, fear, happy, neutral, sad, surprise]


In [2]:
import tensorflow
print(tensorflow.test.is_built_with_cuda())
print(tensorflow.test.is_built_with_gpu_support())

True
True


In [3]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("a")

In [4]:
train_dir = "images/train" #passing the path with training images
test_dir = "images/validation"   #passing the path with testing images

In [5]:
img_size = 48 #original size of the image
_batch_size = 128

In [6]:
"""
Data Augmentation
--------------------------
rotation_range = rotates the image with the amount of degrees we provide
width_shift_range = shifts the image randomly to the right or left along the width of the image
height_shift range = shifts image randomly to up or below along the height of the image
horizontal_flip = flips the image horizontally
rescale = to scale down the pizel values in our image between 0 and 1
zoom_range = applies random zoom to our object
validation_split = reserves some images to be used for validation purpose
"""

train_datagen = ImageDataGenerator(width_shift_range = 0.1,
                                height_shift_range = 0.1,
                                horizontal_flip = True,
                                rescale = 1./255,
                                validation_split = 0.2,)
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [7]:
"""
Applying data augmentation to the images as we read 
them from their respective directories
"""
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (img_size,img_size),
                                                    class_mode = "sparse",
                                                    color_mode = "grayscale",
                                                    subset = "training",
                                                    batch_size=_batch_size
                                                   )
validation_generator = validation_datagen.flow_from_directory( directory = test_dir,
                                                              target_size = (img_size,img_size),
                                                              class_mode = "sparse",
                                                              color_mode = "grayscale",
                                                              subset = "validation",
                                                              batch_size=_batch_size
                                                             )

Found 23060 images belonging to 7 classes.
Found 1411 images belonging to 7 classes.


In [8]:
model= tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(300,activation = 'relu'))
#model.add(Dense(7, activation='softmax'))

In [9]:
label_dict = {0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Neutral',5:'Sad',6:'Surprise'}

In [10]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)

class NumLoss(tf.keras.losses.Loss) :
    def call(self, y_true, y_pred):
        difference = 0.
        for n in range(len(y_true.numpy().tolist())-1) : 
            y_true_n = y_true.numpy().tolist()[n]
            y_pred_n = y_pred.numpy().tolist()[n]
            difference += abs(weight[int(y_true_n[0])] - y_pred_n)
            print(difference)
        return difference / _batch_size

model.compile(
    run_eagerly = True,
    optimizer = Adam(learning_rate=0.0001), 
    loss=NumLoss(),
    metrics=['accuracy']
  )

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 128)       2

In [12]:
epochs = 60
batch_size=_batch_size

In [13]:
print("a")
history = model.fit(x = train_generator, epochs = epochs, validation_data = validation_generator)

a
Epoch 1/60


In [ ]:
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12,4)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
model.save('model_optimal.h5')

In [ ]:
train_loss, train_acc = model.evaluate(train_generator)
test_loss, test_acc   = model.evaluate(validation_generator)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_acc*100, test_acc*100))

In [ ]:
model.save_weights('model_weights.h5')